# Delta Lake Operations - Demo

**Cel szkoleniowy:** Opanowanie podstawowych i zaawansowanych operacji Delta Lake, w tym CRUD, MERGE, Time Travel i optymalizacji.

**Zakres tematyczny:**
- Delta Lake core features: ACID, Delta Log, Schema enforcement
- Schema evolution (additive, automatic)
- Time Travel i Copy-on-write
- CRUD operations: CREATE TABLE, INSERT, UPDATE, DELETE
- MERGE INTO - logika zmian na kluczach
- DESCRIBE DETAIL, DESCRIBE HISTORY
- Optymalizacja: OPTIMIZE, ZORDER BY, VACUUM

## Kontekst i wymagania

- **Dzień szkolenia**: Dzień 2 - Lakehouse & Delta Lake
- **Typ notebooka**: Demo
- **Wymagania techniczne**:
  - Databricks Runtime 13.0+ (zalecane: 14.3 LTS)
  - Unity Catalog włączony
  - Uprawnienia: CREATE TABLE, CREATE SCHEMA, SELECT, MODIFY
  - Klaster: Standard z minimum 2 workers

## Wstęp teoretyczny

**Cel sekcji:** Zrozumienie fundamentów Delta Lake jako formatu tabel transakcyjnych dla data lakehouse.

**Podstawowe pojęcia:**
- **Delta Lake**: Format tabel oparty na Parquet z warstwą transakcyjną (ACID)
- **Delta Log**: Dziennik transakcji przechowujący metadane wszystkich operacji
- **ACID**: Atomicity, Consistency, Isolation, Durability - gwarancje transakcyjne
- **Time Travel**: Możliwość odczytania danych z dowolnego punktu w historii
- **Schema Evolution**: Automatyczne lub kontrolowane zmiany w schemacie tabeli

**Dlaczego to ważne?**
Delta Lake rozwiązuje fundamentalne problemy tradycyjnych data lake'ów: brak transakcji, trudności z aktualizacją danych, brak schema enforcement. Pozwala na reliable data pipelines z gwarancjami ACID i możliwością audytu zmian.

## Izolacja per użytkownik

Uruchom skrypt inicjalizacyjny dla per-user izolacji katalogów i schematów:

In [0]:
%run ../00_setup

## Konfiguracja

Import bibliotek i ustawienie zmiennych środowiskowych:

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window
from datetime import datetime, timedelta

# Wyświetl kontekst użytkownika
print("=== Kontekst użytkownika ===")
print(f"Katalog: {CATALOG}")
print(f"Schema Bronze: {BRONZE_SCHEMA}")
print(f"Schema Silver: {SILVER_SCHEMA}")
print(f"Schema Gold: {GOLD_SCHEMA}")
print(f"Użytkownik: {raw_user}")

# Ustaw katalog i schemat jako domyślne
spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"USE SCHEMA {BRONZE_SCHEMA}")

# Ścieżki do danych źródłowych
CUSTOMERS_CSV = f"{DATASET_BASE_PATH}/customers/customers.csv"
ORDERS_JSON = f"{DATASET_BASE_PATH}/orders/orders_batch.json"

# Nazwy tabel Delta, które będziemy tworzyć
CUSTOMERS_DELTA = f"{BRONZE_SCHEMA}.customers_delta"
ORDERS_DELTA = f"{BRONZE_SCHEMA}.orders_delta"

print(f"\n=== Tabele Delta ===")
print(f"Customers: {CUSTOMERS_DELTA}")
print(f"Orders: {ORDERS_DELTA}")

---

## Sekcja 1: Delta Lake Core Features

**Wprowadzenie teoretyczne:**

Delta Lake dodaje warstwę transakcyjną na top of Parquet files. Każda operacja zapisu (INSERT, UPDATE, DELETE, MERGE) jest zapisywana w Delta Log jako transakcja. Delta Log to seria JSON files w folderze `_delta_log/` zawierająca metadane wszystkich zmian.

**Kluczowe pojęcia:**
- **Transaction Log**: Pojedynczy wpis w Delta Log reprezentujący jedną transakcję
- **Checkpoint**: Snapshots stanu tabeli co N transakcji dla szybszego odczytu
- **Schema Enforcement**: Automatyczna walidacja typów danych przy zapisie
- **Optimistic Concurrency**: Wiele czytających, jeden piszący w tym samym czasie

**Zastosowanie praktyczne:**
- Reliable ETL pipelines z gwarancjami ACID
- Incremental data processing z możliwością rollback
- Audit trails - pełna historia zmian w danych

### Przykład 1.1: Utworzenie tabeli Delta z CSV

**Cel:** Demonstracja utworzenia pierwszej tabeli Delta Lake z danych CSV

**Podejście:**
1. Wczytaj dane z CSV do DataFrame
2. Zapisz DataFrame jako tabelę Delta w Unity Catalog
3. Sprawdź metadane tabeli Delta

In [0]:
# Przykład 1.1 - Utworzenie tabeli Delta z CSV

# Wczytaj dane klientów z CSV
customers_df = (
    spark.read
    .format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(CUSTOMERS_CSV)
)

print(f"Wczytano {customers_df.count()} rekordów")
customers_df.printSchema()

# Zapisz jako tabelę Delta
(
    customers_df
    .write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable(CUSTOMERS_DELTA)
)

print(f"\n✓ Utworzono tabelę Delta: {CUSTOMERS_DELTA}")

**Wyjaśnienie:**

Powyższy kod wykonuje trzy kluczowe operacje:
1. **Wczytanie danych**: Spark DataFrame API wczytuje CSV z automatycznym wykrywaniem schematu
2. **Zapis jako Delta**: `.format("delta")` zapisuje dane w formacie Delta Lake (Parquet + Delta Log)
3. **Rejestracja w Unity Catalog**: `.saveAsTable()` rejestruje tabelę w katalogu UC, co umożliwia governance

Opcja `overwriteSchema=true` pozwala na zmianę schematu przy ponownym zapisie (użyteczne podczas development).

### Przykład 1.2: Inspektowanie Delta Log

**Cel:** Zrozumienie struktury Delta Log i metadanych tabeli

In [0]:
# Przykład 1.2 - DESCRIBE DETAIL

# Wyświetl szczegółowe informacje o tabeli Delta
detail_df = spark.sql(f"DESCRIBE DETAIL {CUSTOMERS_DELTA}")

print("=== DESCRIBE DETAIL ===")
display(detail_df)

# Kluczowe metadane
row = detail_df.collect()[0]
print(f"\n=== Kluczowe metadane ===")
print(f"Format: {row['format']}")
print(f"Lokalizacja: {row['location']}")
print(f"Liczba plików: {row['numFiles']}")
print(f"Rozmiar (bajty): {row['sizeInBytes']}")

### Przykład 1.3: Historia transakcji (DESCRIBE HISTORY)

**Cel:** Przegląd pełnej historii operacji na tabeli Delta

In [0]:
# Przykład 1.3 - DESCRIBE HISTORY

history_df = spark.sql(f"DESCRIBE HISTORY {CUSTOMERS_DELTA}")

print("=== DESCRIBE HISTORY ===")
print("Każdy wiersz reprezentuje jedną transakcję Delta")
display(
    history_df.select(
        "version",
        "timestamp",
        "operation",
        "operationParameters",
        "userName"
    )
)

---

## Sekcja 2: CRUD Operations

**Wprowadzenie teoretyczne:**

Delta Lake wspiera pełne operacje CRUD (Create, Read, Update, Delete) na tabelach. W przeciwieństwie do tradycyjnych data lake'ów (immutable files), Delta Lake umożliwia modyfikację i usuwanie rekordów przy zachowaniu gwarancji ACID.

**Kluczowe operacje:**
- **INSERT**: Dodawanie nowych rekordów (append mode)
- **UPDATE**: Modyfikacja istniejących rekordów na podstawie warunku
- **DELETE**: Usuwanie rekordów spełniających warunek
- **MERGE**: Upsert - INSERT nowych + UPDATE istniejących w jednej transakcji

### Przykład 2.1: INSERT - Dodawanie nowych rekordów

**Cel:** Demonstracja append mode - dodanie nowych klientów do tabeli

**Podejście:**
1. Utworzenie DataFrame z nowymi danymi
2. Append do istniejącej tabeli Delta
3. Weryfikacja wyników

In [0]:
# Przykład 2.1 - INSERT (append)

# Policz rekordy przed
count_before = spark.table(CUSTOMERS_DELTA).count()
print(f"Liczba klientów przed INSERT: {count_before}")

# Utwórz nowe dane do dodania
new_customers_data = [
    ("CUST_9001", "Jan", "Kowalski", "jan.kowalski@example.com", "Poland", "2025-01-15"),
    ("CUST_9002", "Anna", "Nowak", "anna.nowak@example.com", "Poland", "2025-01-16"),
    ("CUST_9003", "Piotr", "Wiśniewski", "piotr.wisniewski@example.com", "Poland", "2025-01-17")
]

new_customers_df = spark.createDataFrame(
    new_customers_data,
    ["customer_id", "first_name","last_name","email", "country", "registration_date"]
)
from pyspark.sql.functions import col; new_customers_df = new_customers_df.select("customer_id","first_name","last_name","email", "country", col("registration_date").cast("date"))

display(new_customers_df)

In [0]:
display(spark.table(CUSTOMERS_DELTA))

In [0]:
# Append nowych klientów
new_customers_df.write.mode("append").saveAsTable(CUSTOMERS_DELTA)

# Policz rekordy po
count_after = spark.table(CUSTOMERS_DELTA).count()
print(f"Liczba klientów po INSERT: {count_after}")
print(f"Dodano: {count_after - count_before} nowych rekordów")

# Sprawdź nowe rekordy
print("\n=== Nowo dodane rekordy ===")
display(
    spark.table(CUSTOMERS_DELTA)
    .filter(F.col("customer_id").isin(["CUST_9001", "CUST_9002", "CUST_9003"]))
)

**Wyjaśnienie:**

Operacja INSERT (append mode) dodaje nowe rekordy bez modyfikacji istniejących. Delta Lake:
- Tworzy nowe pliki Parquet z nowymi danymi
- Dodaje transakcję do Delta Log
- Zachowuje atomicity - albo wszystkie rekordy są dodane, albo żaden

Mode `append` jest najbezpieczniejszy - nie modyfikuje istniejących danych.

### Przykład 2.2: UPDATE - Modyfikacja rekordów

**Cel:** Aktualizacja email dla określonych klientów

In [0]:
# Wyświetl rekordy przed UPDATE
print("=== Przed UPDATE ===")
display(
    spark.table(CUSTOMERS_DELTA)
    .filter(F.col("customer_id") == "CUST_9001")
)

In [0]:
# Przykład 2.2 - UPDATE

from delta.tables import DeltaTable

# Wykonaj UPDATE używając Delta Table API
deltaTable = DeltaTable.forName(spark, CUSTOMERS_DELTA)

deltaTable.update(
    condition = "customer_id = 'CUST_9001'",
    set = { "email": "'jan.kowalski.NEW@example.com'" }
)

In [0]:
print("\n✓ UPDATE wykonany")

# Wyświetl rekordy po UPDATE
print("\n=== Po UPDATE ===")
display(
    spark.table(CUSTOMERS_DELTA)
    .filter(F.col("customer_id") == "CUST_9001")
)

**Wyjaśnienie:**

UPDATE w Delta Lake:
- Nie modyfikuje istniejących plików Parquet (immutable)
- Tworzy nowe pliki z zaktualizowanymi rekordami
- Oznacza stare pliki jako usunięte w Delta Log
- Copy-on-write semantics zapewnia izolację czytających transakcji

Delta Table API (`DeltaTable.forName()`) jest rekomendowanym sposobem wykonywania UPDATE/DELETE/MERGE.

### Przykład 2.3: DELETE - Usuwanie rekordów

**Cel:** Usunięcie określonych klientów

In [0]:
# Policz rekordy przed DELETE
count_before = spark.table(CUSTOMERS_DELTA).count()
print(f"Liczba klientów przed DELETE: {count_before}")

In [0]:
# Przykład 2.3 - DELETE

# Wykonaj DELETE
deltaTable = DeltaTable.forName(spark, CUSTOMERS_DELTA)

(
    deltaTable.delete(
        condition = "customer_id IN ('CUST_9001', 'CUST_9002')"
    )
)


In [0]:
print("\n✓ DELETE wykonany")

# Policz rekordy po DELETE
count_after = spark.table(CUSTOMERS_DELTA).count()
print(f"Liczba klientów po DELETE: {count_after}")
print(f"Usunięto: {count_before - count_after} rekordów")

# Sprawdź, że rekordy zostały usunięte
deleted_check = (
    spark.table(CUSTOMERS_DELTA)
    .filter(F.col("customer_id").isin(['CUST_9001', 'CUST_9002']))
    .count()
)
print(f"\nLiczba rekordów z ID 9002, 9003: {deleted_check} (powinno być 0)")

---

## Sekcja 3: MERGE INTO - Upsert Logic

**Wprowadzenie teoretyczne:**

MERGE INTO to najbardziej potężna operacja Delta Lake, łącząca INSERT i UPDATE w jednej transakcji. Umożliwia implementację upsert logic: "if record exists - update, else - insert". Jest kluczowa dla Slowly Changing Dimensions (SCD) i incremental ETL.

**Kluczowe koncepty:**
- **Source**: DataFrame z nowymi/zmienionymi danymi
- **Target**: Tabela Delta do aktualizacji
- **Merge Key**: Kolumny do identyfikacji dopasowanych rekordów
- **WHEN MATCHED**: Co zrobić z rekordami, które istnieją w obu
- **WHEN NOT MATCHED**: Co zrobić z nowymi rekordami (tylko w source)

### Przykład 3.1: Podstawowy MERGE (Upsert)

**Cel:** Załaduj nowe zamówienia i zaktualizuj istniejące w jednej operacji

**Podejście:**
1. Utworzenie tabeli orders_delta z danych JSON
2. Przygotowanie nowych/zmienionych zamówień
3. MERGE na kluczu order_id

In [0]:
# Przykład 3.1 - MERGE INTO (podstawowy upsert)

# Krok 1: Utworzenie tabeli orders_delta
orders_df = (
    spark.read
    .format("json")
    .option("multiLine", "true")
    .load(ORDERS_JSON)
)

(
    orders_df
    .write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable(ORDERS_DELTA)
)

print(f"✓ Utworzono tabelę: {ORDERS_DELTA}")
count_initial = spark.table(ORDERS_DELTA).count()
print(f"Liczba zamówień (initial): {count_initial}")

In [0]:
# Krok 2: Przygotowanie danych do MERGE
# Symulacja nowych i zmienionych zamówień

updates_data = [
    # Istniejące zamówienie - zmiana total_amount (UPDATE)
    ("ORD00000001", "CUST005909", "PROD000164", "STORE017", "2024-12-31T23:56:00", 
     1, 206.74, 0, 250.00, "Cash"),
    
    # Nowe zamówienie (INSERT)
    ("ORD99999999", "CUST009999", "PROD001234", "STORE050", "2025-01-20T14:30:00", 
     2, 149.99, 10, 269.98, "Credit Card"),
    
    # Kolejne nowe zamówienie (INSERT)
    ("ORD99999998", "CUST009998", "PROD000567", "STORE025", "2025-01-21T10:15:00", 
     3, 66.50, 5, 189.53, "Debit Card")
]

updates_df = spark.createDataFrame(
    updates_data,
    ["order_id", "customer_id", "product_id", "store_id", "order_datetime",
     "quantity", "unit_price", "discount_percent", "total_amount", "payment_method"]
)

print("=== Dane do MERGE ===")
print("Uwaga: Używamy order_datetime (jak w orders_batch.json)")
display(updates_df)

In [0]:
# Krok 3: Wykonanie MERGE

from delta.tables import DeltaTable

deltaTable = DeltaTable.forName(spark, ORDERS_DELTA)

(
    deltaTable.alias("target")
    .merge(
        updates_df.alias("source"),
        "target.order_id = source.order_id"  # Merge key
    )
    .whenMatchedUpdate(set = {
        "total_amount": "source.total_amount",
        "payment_method": "source.payment_method"
    })
    .whenNotMatchedInsert(values = {
        "order_id": "source.order_id",
        "customer_id": "source.customer_id",
        "product_id": "source.product_id",
        "store_id": "source.store_id",
        "order_datetime": "source.order_datetime",
        "quantity": "source.quantity",
        "unit_price": "source.unit_price",
        "discount_percent": "source.discount_percent",
        "total_amount": "source.total_amount",
        "payment_method": "source.payment_method"
    })
    .execute()
)

print("\n✓ MERGE wykonany")

count_after = spark.table(ORDERS_DELTA).count()
print(f"Liczba zamówień (po MERGE): {count_after}")
print(f"Dodano nowych: {count_after - count_initial}")

In [0]:
# Weryfikacja wyników MERGE

print("=== Zamówienie zaktualizowane (order_id = ORD00000001) ===")
display(
    spark.table(ORDERS_DELTA)
    .filter(F.col("order_id") == "ORD00000001")
)

print("\n=== Nowe zamówienia (order_id LIKE 'ORD99999%') ===")
display(
    spark.table(ORDERS_DELTA)
    .filter(F.col("order_id").like("ORD99999%"))
)

**Wyjaśnienie:**

MERGE INTO w Delta Lake:
1. **Merge Key**: `order_id` identyfikuje dopasowane rekordy (klucz biznesowy)
2. **WHEN MATCHED**: Aktualizuje `total_amount` i `payment_method` dla istniejących zamówień
3. **WHEN NOT MATCHED**: Wstawia nowe zamówienia z wszystkimi kolumnami ze schematu orders
4. **Atomicity**: Cała operacja (UPDATE + INSERT) jest jedną transakcją ACID

To najbardziej efektywny sposób implementacji incremental data loading w Delta Lake.

**Zalety MERGE vs DELETE + INSERT:**
- Jedna transakcja zamiast dwóch (szybsze, bezpieczniejsze)
- Brak data loss risk przy failure
- Lepsza performance dla dużych tabel
- Automatyczne partition pruning

---

## Sekcja 4: Time Travel

**Wprowadzenie teoretyczne:**

Time Travel to unikalna funkcja Delta Lake umożliwiająca odczyt danych z dowolnego punktu w historii tabeli. Każda wersja tabeli jest identyfikowana przez:
- **Version number**: Liczba całkowita (0, 1, 2, ...)
- **Timestamp**: Data i czas transakcji

**Kluczowe zastosowania:**
- Audyt zmian w danych
- Rollback do poprzedniego stanu po błędzie
- Reprodukowalność analiz (odczyt danych "as of" określonej daty)
- Porównanie zmian między wersjami

### Przykład 4.1: Odczyt historii wersji

**Cel:** Przegląd wszystkich wersji tabeli i ich metadanych

In [0]:
# Przykład 4.1 - DESCRIBE HISTORY dla Time Travel

# Wyświetl pełną historię tabeli customers_delta
history_df = spark.sql(f"DESCRIBE HISTORY {CUSTOMERS_DELTA}")

print("=== Historia transakcji (DESCRIBE HISTORY) ===")
print("Każda wersja reprezentuje transakcję ACID")

display(
    history_df.select(
        "version",
        "timestamp",
        "operation",
        "operationParameters",
        "userName",
        "operationMetrics"
    )
    .orderBy("version", ascending=False)
)

# Zapisz najnowszą i poprzednią wersję do późniejszego użycia
latest_version = history_df.agg({"version": "max"}).collect()[0][0]
print(f"\nNajnowsza wersja: {latest_version}")

### Przykład 4.2: Odczyt danych z określonej wersji

**Cel:** Użycie Time Travel do odczytania danych z poprzedniej wersji tabeli

In [0]:
# Przykład 4.2 - Odczyt danych @ VERSION AS OF

# Sprawdź liczbę rekordów w najnowszej wersji
current_count = spark.table(CUSTOMERS_DELTA).count()
print(f"Liczba klientów (najnowsza wersja): {current_count}")

# Odczytaj dane z wersji 0 (initial load)
version_0_df = spark.read.format("delta").option("versionAsOf", 0).table(CUSTOMERS_DELTA)
version_0_count = version_0_df.count()

print(f"Liczba klientów (wersja 0): {version_0_count}")
print(f"Różnica: {current_count - version_0_count} rekordów")

print("\n=== Dane z wersji 0 (pierwsze 5 rekordów) ===")
display(version_0_df.limit(5))

# Alternatywnie: odczyt przez SQL
print("\n=== Odczyt przez SQL (wersja 0) ===")
df_sql = spark.sql(f"SELECT * FROM {CUSTOMERS_DELTA} VERSION AS OF 0 LIMIT 5")
display(df_sql)

### Przykład 4.3: Rollback tabeli (RESTORE)

**Cel:** Przywrócenie tabeli do stanu z poprzedniej wersji

**Uwaga:** RESTORE wykonuje nową transakcję, nie usuwa historii!

In [0]:
# Przykład 4.3 - RESTORE TABLE

# Sprawdź aktualną liczbę rekordów
before_restore = spark.table(CUSTOMERS_DELTA).count()
print(f"Liczba klientów przed RESTORE: {before_restore}")

# Przywróć tabelę do wersji 0 (initial load)
spark.sql(f"RESTORE TABLE {CUSTOMERS_DELTA} TO VERSION AS OF 0")

print("\n✓ RESTORE wykonany")

# Sprawdź liczbę rekordów po RESTORE
after_restore = spark.table(CUSTOMERS_DELTA).count()
print(f"Liczba klientów po RESTORE: {after_restore}")

# Sprawdź historię - RESTORE dodaje nową transakcję!
history_after_restore = spark.sql(f"DESCRIBE HISTORY {CUSTOMERS_DELTA}")
print("\n=== Historia po RESTORE ===")
display(
    history_after_restore
    .select("version", "timestamp", "operation", "operationParameters")
    .orderBy("version", ascending=False)
    .limit(3)
)

print("\nUwaga: RESTORE utworzył nową wersję - historia NIE została utracona!")

---

## Sekcja 5: Schema Evolution

**Wprowadzenie teoretyczne:**

Schema Evolution to mechanizm automatycznej adaptacji schematu tabeli Delta do zmian w danych. Delta Lake wspiera:
- **Additive schema changes**: Dodawanie nowych kolumn
- **Schema enforcement**: Blokowanie niekompatybilnych zmian typu
- **Schema merge**: Automatyczne dodawanie kolumn przy zapisie

**Kluczowe opcje:**
- `mergeSchema=true`: Automatyczne dodawanie nowych kolumn z source DataFrame
- `overwriteSchema=true`: Całkowite zastąpienie schematu (uwaga: destructive!)

**Kiedy używać:**
- Dodawanie nowych atrybutów do danych bez przebudowy pipeline'u
- Evolving data models w zgodzie z business requirements
- Integracja nowych źródeł danych z dodatkowymi polami

### Przykład 5.1: Automatyczne dodawanie nowych kolumn

**Cel:** Demonstracja additive schema evolution z opcją `mergeSchema`

In [0]:
# Przykład 5.1 - Schema Evolution z mergeSchema

# KROK 1: Sprawdź aktualny schemat tabeli
print("=== Aktualny schemat tabeli ===")
spark.table(CUSTOMERS_DELTA).printSchema()

# Policz kolumny
current_columns = len(spark.table(CUSTOMERS_DELTA).columns)
print(f"\nAktualna liczba kolumn: {current_columns}")

# Utwórz DataFrame z nowymi kolumnami
extended_customers = spark.createDataFrame([
extended_customers = spark.createDataFrame([
    ("CUST_9999", "Ewa", "Wiśniewska", "ewa.wisn@example.com", "+48 600 123 456",
     "Warsaw", "mazowieckie", "Poland", "2025-01-25", "Premium", "Premium", "PL")
], ["customer_id", "first_name", "last_name", "email", "phone", "city", "state", 
    "country", "registration_date", "customer_segment", "loyalty_tier", "preferred_language"])

print("\n=== Nowy DataFrame z dodatkowymi kolumnami ===")
print("Nowe kolumny: loyalty_tier, preferred_language")
extended_customers.printSchema()

# Zapis bez mergeSchema - spowoduje błąd!
try:
    (
        extended_customers
        .write
        .format("delta")
        .mode("append")
        .saveAsTable(CUSTOMERS_DELTA)
    )
except Exception as e:
    print(f"\n❌ Błąd (oczekiwany): {str(e)[:200]}...")
    print("\nDelta Lake blokuje niezgodne schematy!")

# Zapis z mergeSchema - automatyczne dodanie kolumn
(
    extended_customers
    .write
    .format("delta")
    .mode("append")
    .option("mergeSchema", "true")
    .saveAsTable(CUSTOMERS_DELTA)
)

print("\n✓ Zapis z mergeSchema=true wykonany")

# Sprawdź nowy schemat
print("\n=== Schemat po schema evolution ===")
spark.table(CUSTOMERS_DELTA).printSchema()

# Sprawdź nowy rekord
print("\n=== Nowy rekord z dodatkowymi kolumnami (loyalty_tier, preferred_language) ===")
display(
    spark.table(CUSTOMERS_DELTA)
    .filter(F.col("customer_id") == "CUST_9999")
    .select("customer_id", "first_name", "last_name", "email", "customer_segment", 
            "loyalty_tier", "preferred_language")
)

# Sprawdź stare rekordy - mają NULL w nowych kolumnach
print("\n=== Stare rekordy (mają NULL w loyalty_tier, preferred_language) ===")
display(
    spark.table(CUSTOMERS_DELTA)
    .filter(F.col("customer_id").like("CUST000%"))
    .select("customer_id", "first_name", "last_name", "customer_segment", 
            "loyalty_tier", "preferred_language")
    .limit(5)
)

print("\n⚠️  Uwaga: Stare rekordy mają NULL w nowych kolumnach (loyalty_tier, preferred_language)!")

In [ ]:
# KROK 2: Utwórz DataFrame z nowymi kolumnami (loyalty_tier, preferred_language)
# Używamy istniejącego schematu (first_name, last_name) + nowe kolumny
extended_customers = spark.createDataFrame([
    ("CUST_9999", "Ewa", "Wiśniewska", "ewa.wisn@example.com", "+48 600 123 456",
     "Warsaw", "mazowieckie", "Poland", "2025-01-25", "Premium", "Premium", "PL")
], ["customer_id", "first_name", "last_name", "email", "phone", "city", "state", 
    "country", "registration_date", "customer_segment", "loyalty_tier", "preferred_language"])

print("=== Nowy DataFrame z dodatkowymi kolumnami ===")
print("Nowe kolumny: loyalty_tier, preferred_language")
extended_customers.printSchema()

# Policz kolumny w nowym DataFrame
new_columns = len(extended_customers.columns)
print(f"\nLiczba kolumn w nowym DataFrame: {new_columns}")
print(f"Różnica: +{new_columns - current_columns} nowe kolumny")

In [ ]:
# KROK 3: Zapis bez mergeSchema - spowoduje błąd!
print("=== Próba zapisu bez mergeSchema ===")
try:
    (
        extended_customers
        .write
        .format("delta")
        .mode("append")
        .saveAsTable(CUSTOMERS_DELTA)
    )
    print("✓ Zapis się powiódł (nieoczekiwane!)")
except Exception as e:
    print(f"❌ Błąd (oczekiwany): {str(e)[:200]}...")
    print("\nDelta Lake blokuje niezgodne schematy!")
    print("Powód: Tabela ma {0} kolumn, nowy DataFrame ma {1} kolumn".format(
        current_columns, new_columns
    ))

In [ ]:
# KROK 4: Zapis z mergeSchema - automatyczne dodanie kolumn
print("=== Zapis z mergeSchema=true ===")

(
    extended_customers
    .write
    .format("delta")
    .mode("append")
    .option("mergeSchema", "true")
    .saveAsTable(CUSTOMERS_DELTA)
)

print("✓ Zapis z mergeSchema=true wykonany pomyślnie!")
print("Delta Lake automatycznie dodał nowe kolumny do schematu tabeli")

In [ ]:
# KROK 5: Sprawdź nowy schemat po schema evolution
print("=== Schemat po schema evolution ===")
spark.table(CUSTOMERS_DELTA).printSchema()

# Policz kolumny po evolution
columns_after = len(spark.table(CUSTOMERS_DELTA).columns)
print(f"\nLiczba kolumn PO evolution: {columns_after}")
print(f"Dodano kolumn: {columns_after - current_columns}")
print(f"Nowe kolumny: loyalty_tier, preferred_language")

In [ ]:
# KROK 6: Weryfikacja - nowy rekord z dodatkowymi kolumnami
print("=== Nowy rekord z dodatkowymi kolumnami (loyalty_tier, preferred_language) ===")
display(
    spark.table(CUSTOMERS_DELTA)
    .filter(F.col("customer_id") == "CUST_9999")
    .select("customer_id", "first_name", "last_name", "email", "customer_segment", 
            "loyalty_tier", "preferred_language")
)

# Sprawdź stare rekordy - mają NULL w nowych kolumnach
print("\n=== Stare rekordy (mają NULL w loyalty_tier, preferred_language) ===")
display(
    spark.table(CUSTOMERS_DELTA)
    .filter(F.col("customer_id").like("CUST000%"))
    .select("customer_id", "first_name", "last_name", "customer_segment", 
            "loyalty_tier", "preferred_language")
    .limit(5)
)

print("\n⚠️  Uwaga: Stare rekordy mają NULL w nowych kolumnach (loyalty_tier, preferred_language)!")
print("To jest normalne zachowanie przy additive schema evolution.")
print("Możesz później wypełnić NULL-e używając UPDATE lub fillna()")

---

## Sekcja 6: Optymalizacja tabeli Delta

**Wprowadzenie teoretyczne:**

Delta Lake akumuluje wiele małych plików przy częstych zapisach. Optymalizacja jest kluczowa dla performance:

**OPTIMIZE (Compaction):**
- Łączy małe pliki Parquet w większe (target: 1 GB)
- Zmniejsza overhead odczytu (mniej plików = mniej I/O operations)
- Poprawia query performance

**ZORDER BY:**
- Co-locality algorytm dla multi-dimensional clustering
- Organizuje dane wg często filtrowanych kolumn
- Zmniejsza data skipping overhead

**VACUUM:**
- Usuwa stare pliki Parquet nieużywane przez aktywne wersje
- Zwalnia storage space
- Domyślnie: retention 7 dni (chroni Time Travel)

**Uwaga:** Po VACUUM nie można odczytać starszych wersji poza retention period!

### Przykład 6.1: OPTIMIZE i ZORDER BY

**Cel:** Optymalizacja tabeli orders_delta dla query performance

In [0]:
# Przykład 6.1 - OPTIMIZE i ZORDER BY

# Sprawdź stan tabeli przed OPTIMIZE
detail_before = spark.sql(f"DESCRIBE DETAIL {ORDERS_DELTA}").collect()[0]
print("=== Stan tabeli PRZED OPTIMIZE ===")
print(f"Liczba plików: {detail_before['numFiles']}")
print(f"Rozmiar (bytes): {detail_before['sizeInBytes']}")

# Wykonaj OPTIMIZE
spark.sql(f"OPTIMIZE {ORDERS_DELTA}")
print("\n✓ OPTIMIZE wykonany")

# Sprawdź stan po OPTIMIZE
detail_after = spark.sql(f"DESCRIBE DETAIL {ORDERS_DELTA}").collect()[0]
print("\n=== Stan tabeli PO OPTIMIZE ===")
print(f"Liczba plików: {detail_after['numFiles']}")
print(f"Rozmiar (bytes): {detail_after['sizeInBytes']}")

# OPTIMIZE z ZORDER BY (dla często filtrowanych kolumn)
spark.sql(f"OPTIMIZE {ORDERS_DELTA} ZORDER BY (customer_id, order_datetime)")
print("\n✓ OPTIMIZE ZORDER BY wykonany")

print("\nUwaga: ZORDER BY sortuje dane wg customer_id i order_datetime dla data skipping!")
print("Zapytania z WHERE customer_id = 'X' AND order_datetime >= 'Y' będą znacznie szybsze!")

# Sprawdź historię optymalizacji
optimize_history = spark.sql(f"DESCRIBE HISTORY {ORDERS_DELTA}")
display(
    optimize_history
    .filter(F.col("operation") == "OPTIMIZE")
    .select("version", "timestamp", "operation", "operationMetrics")
)

### Przykład 6.2: VACUUM - czyszczenie starych plików

**Cel:** Usunięcie nieużywanych plików Parquet dla zwolnienia storage

**Ostrzeżenie:** VACUUM usuwa stare pliki - Time Travel będzie ograniczony do retention period!

In [0]:
# Przykład 6.2 - VACUUM

# Sprawdź aktualną liczbę plików
detail_before_vacuum = spark.sql(f"DESCRIBE DETAIL {CUSTOMERS_DELTA}").collect()[0]
print("=== Przed VACUUM ===")
print(f"Liczba plików: {detail_before_vacuum['numFiles']}")

# DRY RUN - pokaż co zostanie usunięte (bez faktycznego usuwania)
print("\n=== VACUUM DRY RUN (retention 0 hours dla demo) ===")
# Uwaga: W produkcji nigdy nie używaj retention 0 hours!
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", "false")

dry_run_result = spark.sql(f"VACUUM {CUSTOMERS_DELTA} RETAIN 0 HOURS DRY RUN")
print("Pliki do usunięcia:")
display(dry_run_result)

# Wykonaj VACUUM (dla demo: 0 hours, w prod: minimum 7 dni)
spark.sql(f"VACUUM {CUSTOMERS_DELTA} RETAIN 0 HOURS")
print("\n✓ VACUUM wykonany")

# Przywróć domyślne ustawienie
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", "true")

# Sprawdź efekt
detail_after_vacuum = spark.sql(f"DESCRIBE DETAIL {CUSTOMERS_DELTA}").collect()[0]
print("\n=== Po VACUUM ===")
print(f"Liczba plików: {detail_after_vacuum['numFiles']}")
print(f"Rozmiar (bytes): {detail_after_vacuum['sizeInBytes']}")

print("\n⚠️  Uwaga: Starsze wersje spoza retention period nie są już dostępne dla Time Travel!")

---

## Best Practices

**Organizacja tabel Delta:**
1. **Naming convention**: Używaj `{layer}.{domain}_{entity}` (np. `bronze.sales_orders`)
2. **Partitioning**: Partycjonuj tylko duże tabele (>1TB) po często filtrowanych kolumnach (np. date)
3. **Table properties**: Ustaw `delta.autoOptimize.optimizeWrite = true` dla częstych małych zapisów

**Optymalizacja performance:**
1. **OPTIMIZE**: Uruchamiaj regularnie (np. co noc) dla tabel z częstymi zapisami
2. **ZORDER BY**: Używaj dla 2-4 najczęściej filtrowanych kolumn (więcej = diminishing returns)
3. **Data skipping**: Wykorzystuj statistics w Delta Log - filtruj po kolumnach z ZORDER

**CRUD operations:**
1. **MERGE**: Preferuj zamiast DELETE + INSERT dla upsert logic (atomicity!)
2. **Merge keys**: Zawsze używaj indeksowanych kolumn (np. primary keys)
3. **Predicates**: Dodawaj dodatkowe predykaty w MERGE dla partition pruning

**Time Travel i maintenance:**
1. **VACUUM retention**: Minimum 7 dni (domyślnie) - chroni Time Travel i concurrent readers
2. **DESCRIBE HISTORY**: Monitoruj operacje i rozmiar Delta Log
3. **Checkpoint**: Tworzony automatycznie co 10 transakcji - nie wymaga interwencji

**Schema evolution:**
1. **mergeSchema**: Używaj ostrożnie - może spowodować NULL-e w starych rekordach
2. **overwriteSchema**: Tylko dla development - destrukcyjna operacja!
3. **NOT NULL constraints**: Definiuj przed pierwszym zapisem (trudno dodać później)

---

## Troubleshooting

**Problem 1: "Schema mismatch" przy zapisie**
```
AnalysisException: A schema mismatch detected when writing to the Delta table
```
**Rozwiązanie:**
- Użyj `option("mergeSchema", "true")` dla additive changes
- Użyj `option("overwriteSchema", "true")` dla pełnej zmiany schematu (uwaga: destructive!)

**Problem 2: "ConcurrentAppendException"**
```
ConcurrentAppendException: Files were added to the table by a concurrent update
```
**Rozwiązanie:**
- Delta Lake używa optimistic concurrency - retry operacji
- Dla częstych konfliktów: zastosuj partition pruning lub MERGE z predykatami

**Problem 3: Time Travel nie działa dla starszych wersji**
```
VersionNotFoundException: Cannot find version X
```
**Rozwiązanie:**
- Sprawdź czy VACUUM nie usunął starych plików
- Zwiększ retention period: `VACUUM table RETAIN 30 DAYS`

**Problem 4: Słaba performance query po wielu UPDATE/DELETE**
**Rozwiązanie:**
- Uruchom `OPTIMIZE` dla compaction małych plików
- Użyj `OPTIMIZE ZORDER BY` dla często filtrowanych kolumn

**Problem 5: Duże zużycie storage pomimo VACUUM**
**Rozwiązanie:**
- Sprawdź `DESCRIBE DETAIL` - czy Delta Log nie rośnie?
- Checkpoint tworzony automatycznie co 10 transakcji
- Rozważ manual checkpoint: `OPTIMIZE table` tworzy checkpoint jako side effect

---

## Podsumowanie

**W tym notebooku nauczyliśmy się:**

✅ **Delta Lake Core Features:**
- Transakcyjna warstwa ACID dla data lakehouse
- Delta Log jako dziennik metadanych wszystkich operacji
- Schema enforcement i automatic schema evolution

✅ **CRUD Operations:**
- INSERT (append mode) - dodawanie nowych rekordów
- UPDATE - modyfikacja istniejących danych z Copy-on-Write semantics
- DELETE - usuwanie rekordów spełniających warunek
- MERGE INTO - upsert logic w jednej transakcji ACID

✅ **Time Travel:**
- Odczyt danych z dowolnej wersji: `VERSION AS OF`, `TIMESTAMP AS OF`
- RESTORE TABLE - rollback do poprzedniego stanu (tworzy nową wersję!)
- DESCRIBE HISTORY - audyt wszystkich operacji na tabeli

✅ **Schema Evolution:**
- Additive changes z `mergeSchema=true`
- Automatyczna adaptacja do nowych kolumn w danych źródłowych
- Schema enforcement chroni przed niekompatybilnymi zmianami

✅ **Optymalizacja:**
- OPTIMIZE - compaction małych plików dla performance
- ZORDER BY - multi-dimensional clustering dla data skipping
- VACUUM - usuwanie nieużywanych plików (uwaga: ogranicza Time Travel!)

**Kluczowe wnioski:**
1. Delta Lake rozwiązuje fundamentalne problemy tradycyjnych data lake'ów (brak transakcji, trudności z UPDATE/DELETE)
2. MERGE INTO jest kluczową operacją dla incremental ETL i Slowly Changing Dimensions
3. Time Travel umożliwia audyt, rollback i reprodukowalność analiz
4. Regularna optymalizacja (OPTIMIZE, VACUUM) jest niezbędna dla production workloads
5. Schema evolution pozwala na evolving data models bez przebudowy pipeline'ów

**Następne kroki:**
- Delta Live Tables - deklaratywne pipeline'y z automatycznym dependency management
- Change Data Feed - incremental processing z CDC patterns
- Delta Sharing - bezpieczne udostępnianie danych bez kopiowania

---

## Cleanup

Opcjonalnie: usuń utworzone tabele Demo po zakończeniu ćwiczeń:

**Uwaga:** Wykonaj cleanup tylko jeśli nie potrzebujesz już tych tabel!

In [0]:
# Cleanup - usuń tabele demo

# Odkomentuj poniższe linie aby usunąć tabele:

# spark.sql(f"DROP TABLE IF EXISTS {CUSTOMERS_DELTA}")
# spark.sql(f"DROP TABLE IF EXISTS {ORDERS_DELTA}")
# print("✓ Tabele usunięte")

print("Cleanup wyłączony (odkomentuj kod aby usunąć tabele)")